# Problems
---
### Shortest Path Problem (Dijkstra's Algorithm)

You have a network of cities represented as a graph. Each connection between cities has a certain distance (or cost). Imagine the graph is implemented as below.

In [ ]:
def dijkstra(graph: dict, start: str, end: str) -> tuple:
    """Dijkstra's algorithm to find the shortest path in a graph."""
    # Initialize distances with infinity for all nodes except the start node
    distances = {node: float('infinity') for node in graph}
    distances[start] = 0
    
    # Dictionary to store the previous node in the optimal path
    previous = {node: None for node in graph}
    
    # Priority queue to get the node with the minimum distance
    priority_queue = [(0, start)]
    
    while priority_queue:
        # Get the node with minimum distance
        priority_queue.sort(key=lambda x: x[0])  # Sort by distance
        current_distance, current_node = priority_queue.pop(0)  # Access the smallest element
        
        # If we've reached the end node, we're done
        if current_node == end:
            break
        
        # If we've found a longer path, skip
        if current_distance > distances[current_node]:
            continue
        
        # Check all neighbors of the current node
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # If we've found a shorter path to the neighbor
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous[neighbor] = current_node
                priority_queue.append((distance, neighbor))
    
    # Reconstruct the shortest path
    path = []
    current = end
    while current:
        path.append(current)
        current = previous[current]
    
    # Return the path in the correct order and the total distance
    return path[::-1], distances[end]

# Define the graph
graph = {
    'A': {'B': 2, 'C': 4},
    'B': {'C': 1, 'D': 7},
    'C': {'D': 3, 'E': 5},
    'D': {'E': 2, 'F': 6},
    'E': {'F': 4},
    'F': {}
}

# Find the shortest path
shortest_path, total_distance = dijkstra(graph, 'A', 'F')

print(f"Shortest path: {' → '.join(shortest_path)}")
print(f"Total distance: {total_distance}")

Shortest path: A → B → C → D → F
Total distance: 12


### Reversing graph
Given a directed graph, reverse the direction of all edges.
1. Create an empty oriented graph,
2. look what `g.vs` and `g.es` give you and use it. (Try to iterate over them).

In [ ]:
import igraph as ig

def reverse_graph(g: ig.Graph):
    """Reverse the direction of all edges in a graph."""
    g_rev = ig.Graph(directed=True)
    for v in g.vs:
        g_rev.add_vertex(name=v["name"])
    for e in g.es:
        g_rev.add_edge(g.vs[e.target]["name"], g.vs[e.source]["name"])
    return g_rev

# Example graph
g = ig.Graph(directed=True)
g.add_vertices(3)
g.add_edges([(0, 1), (1, 2), (2, 0)])
g.vs["name"] = ["A", "B", "C"]

# Usage
print(g)
print(reverse_graph(g))

IGRAPH DN-- 3 3 --
+ attr: name (v)
+ edges (vertex names):
A->B, B->C, C->A
IGRAPH DN-- 3 3 --
+ attr: name (v)
+ edges (vertex names):
B->A, C->B, A->C


# Problematic problems
---
### Multi-criteria Dijkstra's Algorithm
You have a network of cities represented as a graph. Each connection between cities has multiple criteria (e.g., distance, time, cost).

In [10]:
def multi_criteria_dijkstra(graph: dict, start: str, end: str) -> tuple:
    # Initialize distances with infinity for all nodes except the start node
    # For multi-criteria, each distance is now a tuple of values
    criteria_count = len(next(iter(graph[start].values())))  # Get number of criteria
    distances = {node: tuple([float('infinity')] * criteria_count) for node in graph}
    distances[start] = tuple([0] * criteria_count)
    
    # Dictionary to store the previous node in the optimal path
    previous = {node: None for node in graph}
    
    # Priority queue to get the node with the minimum distance
    # For multi-criteria, we use the sum of all criteria as the priority
    priority_queue = [(sum(distances[start]), start)]
    
    while priority_queue:
        # Get the node with minimum summed distance
        priority_queue.sort(key=lambda x: x[0])  # Sort by summed distance
        _, current_node = priority_queue.pop(0)  # Access the smallest element
        
        # If we've reached the end node, we're done
        if current_node == end:
            break
            
        # Check all neighbors of the current node
        for neighbor, weights in graph[current_node].items():
            # Calculate new distances for each criterion
            new_distances = tuple(d + w for d, w in zip(distances[current_node], weights))
            
            # Compare if this path is better - we consider a path better if it's better in at least one criterion and not worse in any
            current_better = False
            new_better = False
            
            for i in range(criteria_count):
                if new_distances[i] < distances[neighbor][i]:
                    new_better = True
                elif new_distances[i] > distances[neighbor][i]:
                    current_better = True
            
            # If new path is better in at least one criterion and not worse in any
            if new_better and not current_better:
                distances[neighbor] = new_distances
                previous[neighbor] = current_node
                # Use sum of criteria for priority queue
                priority_queue.append((sum(new_distances), neighbor))
    
    # Reconstruct the shortest path
    path = []
    current = end
    while current:
        path.append(current)
        current = previous[current]
    
    # Return the path in the correct order and the total distances for each criterion
    return path[::-1], distances[end]

# Define the graph with multiple criteria per edge
# Each edge weight is now a tuple: (distance, time, cost)
graph = {
    'A': {'B': (2, 3, 10), 'C': (4, 1, 5)},
    'B': {'C': (1, 2, 3), 'D': (7, 4, 8)},
    'C': {'D': (3, 2, 6), 'E': (5, 3, 9)},
    'D': {'E': (2, 1, 2), 'F': (6, 3, 7)},
    'E': {'F': (4, 2, 4)},
    'F': {}
}

# Find the shortest path considering all criteria
shortest_path, total_values = multi_criteria_dijkstra(graph, 'A', 'F')

print(f"Shortest path: {' → '.join(shortest_path)}")
print(f"Total values: Distance = {total_values[0]}, Time = {total_values[1]}, Cost = {total_values[2]}")

Shortest path: A → C → D → E → F
Total values: Distance = 13, Time = 6, Cost = 17
